# Pattern Associator

We recreate the network of the [Pattern Associator tutorial from the CECN1 notebook](https://grey.colorado.edu/CompCogNeuro/index.php/CECN1_Pattern_Associator) exploring how the delta rule works and behave. 

In [1]:
import numpy as np
import dotdot
import leabra
import graphs

<IPython.core.display.Javascript object>

In [2]:
import random
random.seed(0)

In [49]:
inputsize = 10
outputsize = 10

In [20]:
def generate_hot_vec(size, hots):
    retval = np.zeros(size)
    for i in range(hots):
        ix = 0
        while True:
            ix = random.randint (0, size-1)
            if retval[ix] != 1.0:
                break
        retval[ix] = 1.0

    return retval

In [43]:
def is_in_matrix(vec, matrix):
    ''' vec - row vector to test
    matrix - matrix possibly containing row vector
    '''
    for i in range(matrix.shape[0]):
        if np.array_equal(vec, matrix[i]): return True
    return False


In [44]:
def generate_unique_patterns(rows, cols, hots):
    retval = np.zeros((rows, cols))
    for j in range(rows):
        vec = np.zeros(cols)
        while True:
            vec = generate_hot_vec(cols, hots)
            if not is_in_matrix(vec, retval): break
        retval[j] = vec
    return retval

In [47]:
generate_unique_patterns(4, 5, 2)

array([[1., 0., 0., 0., 1.],
       [0., 0., 0., 1., 1.],
       [1., 0., 1., 0., 0.],
       [1., 1., 0., 0., 0.]])

In [3]:
u_spec = leabra.UnitSpec(act_thr=0.5, act_gain=100, act_sd=0.01, 
                         g_bar_e=1.0, g_bar_i=1.0, g_bar_l=0.1, 
                         e_rev_e=1.0, e_rev_i=0.25, e_rev_l=0.3,
                         avg_l_min=0.1, avg_l_init=0.4,
                         adapt_on=False)

In [50]:
input_layer  = leabra.Layer(inputsize, unit_spec=u_spec, name='input_layer')
output_spec  = leabra.LayerSpec(g_i=1.5, ff=1.0, fb=0.5, fb_dt=1/1.4, ff0=0.1)
output_layer = leabra.Layer(outputsize, spec=output_spec, unit_spec=u_spec, name='output_layer')

In [51]:
conspec = leabra.ConnectionSpec(proj='full', lrule='leabra', lrate=0.04)
conn    = leabra.Connection(input_layer, output_layer, spec=conspec)

In [53]:
conn.weights

array([[0.28598584, 0.40053077, 0.46803432, 0.28052122, 0.48356561,
        0.54824246, 0.59966156, 0.4456381 , 0.38006663, 0.70219932],
       [0.48510808, 0.70115212, 0.53488427, 0.59884854, 0.35170734,
        0.63367417, 0.64432415, 0.32910431, 0.33097704, 0.51473711],
       [0.30860642, 0.7107092 , 0.58280291, 0.25660188, 0.59064034,
        0.70004903, 0.68740235, 0.70875556, 0.57446674, 0.44432112],
       [0.5788081 , 0.32670639, 0.59541138, 0.4789771 , 0.28953689,
        0.61950806, 0.52215975, 0.3174188 , 0.63108333, 0.49091354],
       [0.55506783, 0.58670173, 0.54513884, 0.69597177, 0.67688719,
        0.31617214, 0.40514879, 0.62424294, 0.66445139, 0.29036155],
       [0.54728846, 0.59929134, 0.33003989, 0.36154891, 0.47406766,
        0.60517499, 0.5868876 , 0.68726882, 0.26577274, 0.6858441 ],
       [0.53373617, 0.63609286, 0.60450302, 0.33283749, 0.28194315,
        0.60075808, 0.47318236, 0.69247275, 0.70401993, 0.55198869],
       [0.25003458, 0.26945509, 0.4129719

In [54]:
network = leabra.Network(layers=[input_layer, output_layer], connections=[conn])

In [55]:
def event(k, network):
    """Run a minus phase and a plus phase for a given input/output pair"""
    inputs  = np.zeros(inputsize) #[0.0, 0.0, 0.0, 0.0]
    outputs = np.zeros(outputsize) # [0.0, 0.0]
    inputs[k] = 1.0
    outputs[int(k/2)] = 1.0  # desired output

    network.set_inputs({'input_layer': inputs})
    network.set_outputs({'output_layer': outputs})

    # minus phase
    for _ in range(3):
        network.quarter()
        print('g_e', [u.g_e for u in output_layer.units])
        print('v_m', [u.v_m for u in output_layer.units])
        print('v_m_eq', [u.v_m_eq for u in output_layer.units])
        print('act', output_layer.activities)
#        print(input_layer.activities)
        print([u.avg_s_eff for u in input_layer.units])
        
    error = sum((np.array(output_layer.activities) - outputs)**2) 

    # plus phase: the output is set directly
    network.quarter()
    
    return error

In [ ]:
conn.weights
print(output_layer.to_connections)

In [ ]:
event(0, network)

In [ ]:
conn.weights

In [ ]:
def trial():
    sse = 0.0
    sse += event(0, network)
    sse += event(1, network)
    sse += event(2, network)
    sse += event(3, network)
    return sse / 4

In [ ]:
err = [trial() for _ in range(20)]

In [ ]:
conn.weights

In [62]:
graphs.line(range(len(err)), err, title="Average error over trials", width=600)

In [63]:
def event_vec(inputs, outputs, network):
    network.set_inputs({'input_layer': inputs})
    network.set_outputs({'output_layer': outputs})

    # minus phase
    for _ in range(3):
        network.quarter()
        # print('g_e', [u.g_e for u in output_layer.units])
        # print('v_m', [u.v_m for u in output_layer.units])
        # print('v_m_eq', [u.v_m_eq for u in output_layer.units])
        # print('act', output_layer.activities)
        # print(input_layer.activities)
        # print([u.avg_s_eff for u in input_layer.units])
        
    error = sum((np.array(output_layer.activities) - outputs)**2) 

    # plus phase: the output is set directly
    network.quarter()
    
    return error

In [65]:
def trial_vec(inputs, outputs):
    sse = 0.0
    for i in range(inputs.shape[0]):
        sse += event_vec(inputs[i], outputs[i], network)
    return sse / inputs.shape[0]


In [74]:
 rows = 10
 cols = inputsize
 hots = 2
 inp = generate_unique_patterns(rows, cols, hots)
 outp = generate_unique_patterns(rows, cols, hots)
 err = [trial_vec(inp, outp) for _ in range(100)]

In [75]:
graphs.line(range(len(err)), err, title="Average error over trials", width=600)